In [ ]:
%load_ext autoreload
%autoreload 2
from helper import *

#### Load data 

In [ ]:
no1_data = pd.read_csv('./data/time_series_60min_singleindex_filtered.csv', usecols=['timestamp', 'NO_1_price_day_ahead', 'NO_1_load_actual_entsoe_transparency', 'NO_1_load_forecast_entsoe_transparency']).fillna(0)
leogo_forecast_data = pd.read_csv('./data/leogo_profiles_forecasts.csv', usecols=[
    'timestamp', 'curve_wind'])  # 30 dager forecast
leogo_nowcast_data = pd.read_csv('./data/leogo_profiles_nowcasts.csv', usecols=[
    'timestamp', 'curve_wind'])  # 30 days nowcast

# Set demand to 30 over all timepoints for now
leogo_demand_data = [30 for _ in range(200)]
my_price_data = no1_data['NO_1_price_day_ahead'] 

# Wind data 
forecast_wind_data = prepare_leogo_forecast_hourly(leogo_forecast_data,0, 31)
nowcast_wind_data = prepare_leogo_forecast_hourly(leogo_nowcast_data ,0, 31)

leogo_turbine_data_json = pd.read_json('./data/gas_turbine_data.json')
leogo_turbine_data = pd.json_normalize(leogo_turbine_data_json["devices"], 
                                       meta=leogo_turbine_data_json.devices)
leogo_turbine_data.set_index("id", inplace=True)

In [ ]:
d_Data = generate_variable_demand()
ax = leogo_nowcast_data[:160].plot(legend=False)
ax2 = leogo_nowcast_data[:160].plot(ax=ax)

Model for the deterministic from thesis, for every prediction window N
$$
\begin{gather}
    \begin{aligned}
        \min_{\bf{x}, \bf{u}} \qquad \sum_{t = 0}^{N}& \left[C^{\rm gt}(t) - C^{\rm reward}(t)+ C^{\rm grid}(t) + C^{\rm CO2}(t) \right]\\
    \text{s.t.} \qquad C^{\rm gt}(t) &= a + bP_{g}(t) + cP_{g}^2(t) & \forall g \in G, t=0,1,\ldots,N\\
    C^{\rm CO2}(t) &= \pi \sum_{g \in G}e_{g}(t) &t=0,1,\ldots,N\\
        C^{\rm grid}(t) &= p(t)P^{\rm grid}(t)  &t=0,1,\ldots,N \\
        C^{\rm reward}(t) &= \theta P^{\rm bess}(t) &t=0,1,\ldots,N\\
        \rho e_g(t)
            &= A P_{g}(t) + B\overline{P_{g}} & \forall g \in G, t=0,1,\ldots,N \\
        P^{\rm bess}(t) &= \left( \eta_c P_{c}(t) - \frac{P_d(t)}{\eta_d} \right) &t=0,1,\ldots,N\\
        E(t+1) &= E(t) +P^{\rm bess}(t) & t=0,1,\ldots, N-1\\
        D(t) &= -P^{\rm bess}(t) + P^{\rm gen}(t) + P^{\rm wind}(t) + P^{\rm grid}(t) &t=0,1,\ldots,N\\
    P^{\rm gen}(t) &= \sum_{g \in G} P_{g}(t) &t=0,1,\ldots,N\\
        \underline{P_{g}} &\leq P_{g}(t) \leq \overline{P_{g}} & \forall g \in G, t=0,1,\ldots,N\\
        \underline{E} &\leq E(t) \leq \overline{E} & t=0,1,\ldots,N \\
    0   &\leq P_c(t) \leq \overline{P_c} &t=0,1,\ldots,N \\
    0 &\leq P_d(t) \leq \overline{P_d} & t=0,1,\ldots,N 
    \end{aligned}
\end{gather}
$$
where 
$$
x(t)=\left[E(t), P^{\rm grid}(t), P^{\rm gt}(t)\right]^T, u(t) = \left[ \textbf{P}(t)\right]^T. 
$$

## Deterministic implementation
- Can set variable demand flag, and if the model should include grid support

In [ ]:
use_variable_demand = True
include_grid = True
demand_data = d_Data if use_variable_demand else leogo_demand_data

def hybrid_solve_deterministic(price_data, wind_data, demand_data, turbine_data, horizon_length=120, include_grid=include_grid, verbose=False):
    # Solve one iteration of the model
    m_gas = build_dg_model(price_data, wind_data, demand_data, turbine_data,
                        horizon_length=horizon_length,include_grid=include_grid)
    # Specify the solver
    solver = pyo.SolverFactory('gurobi')

    # Fix the initial energy amount
    m_gas.E0 = 4.0
    m_gas.vChargeRate[0].fix()
    m_gas.vDischargeRate[0].fix()

    m_gas_results = solver.solve(m_gas, tee=verbose, report_timing=verbose)
    return m_gas, m_gas_results

m_show, _ = hybrid_solve_deterministic(my_price_data, nowcast_wind_data, demand_data, 
                                      leogo_turbine_data, horizon_length=120, include_grid=include_grid, verbose=False)

##### Plotting the resulting model

In [ ]:
df_gas = hybrid_get_deterministic_dataframe(m_show)
# Drawing the plot and subplots
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(23, 12))

P_plot = df_gas['P_gt'].plot(ax=axes[0,0], ylabel='Power generated (MWh)')
chr_plot = df_gas[['vChargeRate', 'vDischargeRate']].plot(drawstyle="steps", ax=axes[0,1], ylabel='Charging rate (MWh)')
chr_plot.axhline(y=0, color='r', linestyle=':')

df_gas['P_wind_output'].plot(ax=axes[1,0], ylabel='Wind power (MWh)')
df_gas['E'].plot(ax=axes[1,1], ylabel='Battery charge')
df_gas['pDemand'].plot(ax=axes[2,0], ylabel="Total power demand (MWh)")
df_gas['CO2'].plot(ax=axes[2,1], ylabel='Total Co2 emissions')
try:
    df_gas['vGrid'].plot(ax=axes[3,0], ylabel='Power used from grid')
except:
    print("No data for grid")
df_gas['vPenalty'].plot(ax=axes[3,1], color="red", ylabel="Penalty")
df_gas['Cumulative Penalty'].plot(ax=axes[4,1], color="black", ylabel="Cumulative penalty")
df_gas['Cumulative Penalty'].iloc[-1]

## Receding horizon controller (pyomo.mpc library)
- Builds model from helper.py
- Converts Horizon to time series, forecast/nowcast data from LEOGO dataset

The ```f_data``` and ```n_data``` can be expanded to most parameters, even the switching on/off of generators can be implemented    
Different penalties that are mutable can be tested in this framework in a EMPC fashion, mut MVP is wind-data

In [ ]:
model_horizon = 10
simulation_steps = 120
nowcast_horizon  = 3        # Refreshing data often 
energy_amount = 4.0

use_variable_demand = True
include_grid = True
demand_data = d_Data if use_variable_demand else leogo_demand_data

- Observe that the model runs and recedes the horizon
- The data should shift right by one for each iteration when applied to forecast data

In [ ]:
def get_steady_state_data(target, tee=False):
    """Get the steady state for the MPC"""
    m_gas = build_dg_model(my_price_data, nowcast_wind_data, demand_data, leogo_turbine_data,
                     horizon_length=model_horizon, include_grid=include_grid)
    interface = mpc.DynamicModelInterface(m_gas, m_gas.N)

    m_gas.E0 = target
    m_gas.vChargeRate[0].fix()
    m_gas.vDischargeRate[0].fix()

    solver = pyo.SolverFactory("gurobi")
    solver.solve(m_gas, tee=tee)
    return interface.get_data_at_time(0)

init_data = get_steady_state_data(target=energy_amount)

f_data = {
        "pWind[*]": [x for x in forecast_wind_data[:simulation_steps]],
        "pPrice[*]": [x for x in my_price_data[:simulation_steps]],
        "pDemand[*]": [x for x in d_Data[:simulation_steps]],
}

m_normal = build_dg_model_MPC(my_price_data, forecast_wind_data, d_Data, leogo_turbine_data,
                     horizon_length=model_horizon, include_grid=include_grid)

m_mpc_gt, mpc_gt_res = run_hybrid_rhc3(m_normal, model_horizon, f_data, f_data, simulation_steps, init_data, nowcast_horizon=nowcast_horizon, verbose=True)

In [ ]:
df_res = pd.DataFrame(*mpc_gt_res.to_serializable())

print(df_res)
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12))
df_penalty = hybrid_get_deterministic_dataframe(m_show)

df_penalty_mpc =  df_res["vPenalty[*]"]
df_cumsum_mpc = df_penalty_mpc.cumsum()
df_penalty_mpc.plot(ax=axes[0], color="red", ylabel="Penalty")
df_cumsum_mpc.plot(ax=axes[1],color="red", ylabel="Cumulative penalty")
df_penalty['Cumulative Penalty'].plot(ax=axes[1],color="black", ylabel="Cumulative penalty")

print(df_cumsum_mpc.iloc[-1])
print(df_penalty["Cumulative Penalty"].iloc[-1])

## Run Hybrid Model using MPC

In [ ]:
f_data = {
        "pWind[*]": [x for x in forecast_wind_data[:simulation_steps]],
        "pPrice[*]": [x for x in my_price_data[:simulation_steps]],
        "pDemand[*]": [x for x in d_Data[:simulation_steps]],
}
n_data = {
        "pWind[*]": [x for x in nowcast_wind_data[:simulation_steps]],
        "pPrice[*]": [x for x in my_price_data[:simulation_steps]],
        "pDemand[*]": [x for x in d_Data[:simulation_steps]],
}

m_prime = build_dg_model_MPC(my_price_data, forecast_wind_data, d_Data, leogo_turbine_data,
                     horizon_length=model_horizon, include_grid=True)

m_mpc_prime, mpc_gt_res_prime= run_hybrid_rhc3(m_prime, model_horizon, f_data, n_data, simulation_steps, init_data, nowcast_horizon=nowcast_horizon, verbose=False)

#### Plotting the instances
- MPC solved solution
- Forecast solution applied to the historic data that occured
- Actual deterministic optimal (hypothetic)

In [ ]:
m_forecast, _ = hybrid_solve_deterministic(my_price_data, forecast_wind_data, demand_data, 
                                      leogo_turbine_data, horizon_length=120, include_grid=include_grid)
m_actual, _ = hybrid_solve_deterministic(my_price_data, nowcast_wind_data, demand_data, 
                                      leogo_turbine_data, horizon_length=120, include_grid=include_grid)
# Extract results
df_res_mpc= pd.DataFrame(*mpc_gt_res_prime.to_serializable())
df_penalty_mpc_nowcast =  df_res_mpc["vPenalty[*]"]
df_cumsum_mpc_nowcast = df_penalty_mpc_nowcast.cumsum()
print(df_res_mpc)

df_penalty = hybrid_get_deterministic_dataframe(m_forecast) 
df_cumsum = df_penalty['Cumulative Penalty']
df_forecast = calculate_hybridmodel_actualcost(df_penalty, my_price_data[:120])
df_actual = hybrid_get_deterministic_dataframe(m_actual)


df_penalty_mpc_nowcast.plot(color="green", ylabel="Penalty", linestyle='-',  alpha=0.8) # mpc
df_penalty['vPenalty'].plot(color="blue", ylabel="Penalty", linestyle='--', alpha=0.8) # forecast 
df_actual['vPenalty'].plot(color="red", ylabel="Penalty",  alpha=0.8) # Acutal 

print(f" MPC total optimal: {df_cumsum_mpc_nowcast.iloc[-1]}")
print(f" Forecast total optimal: {df_cumsum.iloc[-1]}")
print(f" Actual total optimal: {df_actual['Cumulative Penalty'].iloc[-1]}")